In [ ]:
import torch
from torch import Tensor
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image
from pathlib import Path
import numpy

# 필요한 메트릭 및 모델 관련 라이브러리 임포트
from neuralcompression.metrics import MultiscaleStructuralSimilarity, calc_psnr, pickle_size_of
from torchmetrics.image import LearnedPerceptualImagePatchSimilarity

def rescale_image(image: Tensor) -> Tensor:
    """텐서를 [0, 1] 범위에서 [0, 255] 범위의 정수형으로 변환합니다."""
    return (image * 255.0 + 0.5).clamp(0, 255).to(torch.uint8)

# --- 설정 ---
input_path = Path("/workspace/Weight_compression/comp_lm_qtip/nic_models/illm/IMG_0258f.jpg")
output_path = Path("/workspace/Weight_compression/comp_lm_qtip/nic_models/illm/compressed_image.png")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. 모델 불러오기
print("Loading MS-ILLM model...")
model = torch.hub.load("facebookresearch/NeuralCompression", "msillm_quality_6")
model = model.to(device)
model.eval()
model.update(force=True)

# 2. 이미지 불러오기 및 전처리
print(f"Processing image: {input_path}")
image_pil = Image.open(input_path).convert("RGB")
image_tensor = ToTensor()(image_pil).unsqueeze(0).to(device)

# 3. 압축 및 복원 실행
with torch.no_grad():
    compressed_obj = model.compress(image_tensor)
    decompressed_tensor = model.decompress(compressed_obj).clamp(0.0, 1.0)

# 4. 결과 분석 (메트릭 계산)
num_bytes = pickle_size_of(compressed_obj)
bpp = (num_bytes * 8) / (image_tensor.shape[2] * image_tensor.shape[3])

original_rescaled = rescale_image(image_tensor)
decompressed_rescaled = rescale_image(decompressed_tensor)

psnr = calc_psnr(original_rescaled, decompressed_rescaled)
msssim_metric = MultiscaleStructuralSimilarity(data_range=255.0).to(device)
msssim = msssim_metric(decompressed_rescaled, original_rescaled)
lpips_metric = LearnedPerceptualImagePatchSimilarity(normalize=True).to(device)
lpips = lpips_metric(decompressed_tensor, image_tensor)

# 5. 복원된 이미지 저장
output_image_pil = ToPILImage()(decompressed_tensor.squeeze(0).cpu())
output_image_pil.save(output_path)
print(f"Decompressed image saved to: {output_path}")

# 6. 최종 결과 출력
print("\n--- Compression Results ---")
print(f"Rate (bpp): {bpp:.4f}")
print(f"PSNR: {psnr:.4f}")
print(f"MS-SSIM: {msssim.item():.4f}")
print(f"LPIPS: {lpips.item():.4f}")

In [ ]:
model = torch.hub.load("facebookresearch/NeuralCompression", "msillm_quality_3")

In [ ]:
import matplotlib.pyplot as plt

# 원본과 결과 이미지 열기
original_img = Image.open("test_image.png")
compressed_img = Image.open("compressed_image.png")

# 두 이미지를 나란히 표시
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(original_img)
axes[0].set_title(f"Original ({original_img.width}x{original_img.height})")
axes[0].axis("off")

axes[1].imshow(compressed_img)
axes[1].set_title(f"Compressed ({compressed_img.width}x{compressed_img.height})")
axes[1].axis("off")

plt.tight_layout()
plt.show()